# State Farm Distracted Driver Detection


[State Farm Distracted Driver Detection](https://www.kaggle.com/c/state-farm-distracted-driver-detection#evaluation)

## Action Plan
### 1. Data Preparation and Preprocessing
### 2. Finetune and Train Model
### 2b. Data Augmentation parameter tuning
### 3. Generate and Validate Predictions 
### 4. Submit predictions to Kaggle

# 2. Finetune and Train Model

## Setup 

In [17]:
#%cd "~/kaggle/state-farm-driver-detection/code"
%pwd

'D:\\Dropbox\\Ted 2017\\11A_技術\\[TREND] Data Science and ML\\[Tutorials] Kaggle\\[Competitions] Kaggle Competitions\\2016-04_ImgClass_State Farm Distracted Driver Detection\\Code'

In [18]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
CODE_HOME_DIR = current_dir
DATA_HOME_DIR = CODE_HOME_DIR + '/../input/'
print(CODE_HOME_DIR)
print(DATA_HOME_DIR)

D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-04_ImgClass_State Farm Distracted Driver Detection\Code
D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-04_ImgClass_State Farm Distracted Driver Detection\Code/../input/


In [19]:
#import modules
from importlib import reload

import utils; reload(utils)
from utils import *

import vgg16bn_ted
from vgg16bn_ted import Vgg16BN; 

%matplotlib inline

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Setup Paths

In [21]:
#%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' # + '/sample/' 
results_path = path + 'results/'
train_path = path + 'train/'
valid_path = path + 'valid/'
test_path = path + 'test/'
model_path = path + 'models/'
if not os.path.exists(model_path): os.makedirs(model_path)
    
batch_size = 16 #batch_size = 64

#### Setup and save Data Classes, Labels, and Filenames

In [22]:
(trn_classes, val_classes, trn_labels, val_labels, trn_filenames, val_filenames, test_filenames) = get_classes(path)

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
trn_data = load_array(results_path+'trn_data.bc')
val_data = load_array(results_path+'val_data.bc')

## Method 1: First training by Resnet model